In [1]:
# Import library
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
import matplotlib.pyplot as plt # plotting library
import seaborn as sns # visualization library based on matplotlib
from IPython.display import display # Manage multiple output per cell
import datetime
import sqlite3

In [2]:
# Import sickit methods
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, log_loss, accuracy_score, roc_auc_score
from sklearn.calibration import calibration_curve
from sklearn.cross_validation import KFold

/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# Import model
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [4]:
# Import and define deep learning model
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
def create_DNN_model(optimizer,nb_feature):
    model = Sequential()
    model.add(Dense(nb_feature, input_dim=nb_feature, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
    return model
def create_DNN_model_adam_all():
    #return create_DNN_model('adam',184)
    return create_DNN_model('adam',176)
def create_DNN_model_adam_38():
    return create_DNN_model('adam',38)
def create_DNN_model_sgd_all():
    #return create_DNN_model('SGD',184)
    return create_DNN_model('SGD',176)
def create_DNN_model_sgd_56():
    return create_DNN_model('SGD',56)

Using TensorFlow backend.


In [5]:
# Define target
target = 'INFO_FTR_H'
bet_on = 'H'
odd_avg = 'INFO_BbAvH'
odd_pinnacle = 'INFO_PSH'
# Define if we want to train only a specific league
leagues = ['D1', 'E0', 'E1', 'E2', 'F1', 'I1', 'SP1', 'SC0']
leagues = ['D1']
# Define number of fold use for out of fold prediction
NFOLDS = 4
# Season to Test
test_seasons = [2013, 2014]
#test_seasons = [2013]
#test_seasons = [2014]
# Numbers of season for training
#nb_train_seasons = [5, 3, 1]
nb_train_seasons = [5]
# List informative column in dataset
column_info = ['INFO_Date','INFO_Div','INFO_HomeTeam','INFO_AwayTeam','INFO_FTHG','INFO_FTAG','INFO_FTR','INFO_BbAvH','INFO_PSH']

In [6]:
# Define all different features dataset for layer 1
raw_features = ["A_MEANS_FIVE_AC","A_MEANS_FIVE_AF","A_MEANS_FIVE_AR","A_MEANS_FIVE_AS","A_MEANS_FIVE_AST","A_MEANS_FIVE_AY","A_MEANS_FIVE_FTAG","A_MEANS_FIVE_FTHG","A_MEANS_FIVE_FTR_A","A_MEANS_FIVE_FTR_D","A_MEANS_FIVE_FTR_H","A_MEANS_FIVE_HC","A_MEANS_FIVE_HF","A_MEANS_FIVE_HR","A_MEANS_FIVE_HS","A_MEANS_FIVE_HST","A_MEANS_FIVE_HTAG","A_MEANS_FIVE_HTHG","A_MEANS_FIVE_HTR_A","A_MEANS_FIVE_HTR_D","A_MEANS_FIVE_HTR_H","A_MEANS_FIVE_HY","H_MEANS_FIVE_AC","H_MEANS_FIVE_AF","H_MEANS_FIVE_AR","H_MEANS_FIVE_AS","H_MEANS_FIVE_AST","H_MEANS_FIVE_AY","H_MEANS_FIVE_FTAG","H_MEANS_FIVE_FTHG","H_MEANS_FIVE_FTR_A","H_MEANS_FIVE_FTR_D","H_MEANS_FIVE_FTR_H","H_MEANS_FIVE_HC","H_MEANS_FIVE_HF","H_MEANS_FIVE_HR","H_MEANS_FIVE_HS","H_MEANS_FIVE_HST","H_MEANS_FIVE_HTAG","H_MEANS_FIVE_HTHG","H_MEANS_FIVE_HTR_A","H_MEANS_FIVE_HTR_D","H_MEANS_FIVE_HTR_H","H_MEANS_FIVE_HY","A_MEANS_THREE_AC","A_MEANS_THREE_AF","A_MEANS_THREE_AR","A_MEANS_THREE_AS","A_MEANS_THREE_AST","A_MEANS_THREE_AY","A_MEANS_THREE_FTAG","A_MEANS_THREE_FTHG","A_MEANS_THREE_FTR_A","A_MEANS_THREE_FTR_D","A_MEANS_THREE_FTR_H","A_MEANS_THREE_HC","A_MEANS_THREE_HF","A_MEANS_THREE_HR","A_MEANS_THREE_HS","A_MEANS_THREE_HST","A_MEANS_THREE_HTAG","A_MEANS_THREE_HTHG","A_MEANS_THREE_HTR_A","A_MEANS_THREE_HTR_D","A_MEANS_THREE_HTR_H","A_MEANS_THREE_HY","H_MEANS_THREE_AC","H_MEANS_THREE_AF","H_MEANS_THREE_AR","H_MEANS_THREE_AS","H_MEANS_THREE_AST","H_MEANS_THREE_AY","H_MEANS_THREE_FTAG","H_MEANS_THREE_FTHG","H_MEANS_THREE_FTR_A","H_MEANS_THREE_FTR_D","H_MEANS_THREE_FTR_H","H_MEANS_THREE_HC","H_MEANS_THREE_HF","H_MEANS_THREE_HR","H_MEANS_THREE_HS","H_MEANS_THREE_HST","H_MEANS_THREE_HTAG","H_MEANS_THREE_HTHG","H_MEANS_THREE_HTR_A","H_MEANS_THREE_HTR_D","H_MEANS_THREE_HTR_H","H_MEANS_THREE_HY","A_STD_FIVE_AC","A_STD_FIVE_AF","A_STD_FIVE_AR","A_STD_FIVE_AS","A_STD_FIVE_AST","A_STD_FIVE_AY","A_STD_FIVE_FTAG","A_STD_FIVE_FTHG","A_STD_FIVE_FTR_A","A_STD_FIVE_FTR_D","A_STD_FIVE_FTR_H","A_STD_FIVE_HC","A_STD_FIVE_HF","A_STD_FIVE_HR","A_STD_FIVE_HS","A_STD_FIVE_HST","A_STD_FIVE_HTAG","A_STD_FIVE_HTHG","A_STD_FIVE_HTR_A","A_STD_FIVE_HTR_D","A_STD_FIVE_HTR_H","A_STD_FIVE_HY","H_STD_FIVE_AC","H_STD_FIVE_AF","H_STD_FIVE_AR","H_STD_FIVE_AS","H_STD_FIVE_AST","H_STD_FIVE_AY","H_STD_FIVE_FTAG","H_STD_FIVE_FTHG","H_STD_FIVE_FTR_A","H_STD_FIVE_FTR_D","H_STD_FIVE_FTR_H","H_STD_FIVE_HC","H_STD_FIVE_HF","H_STD_FIVE_HR","H_STD_FIVE_HS","H_STD_FIVE_HST","H_STD_FIVE_HTAG","H_STD_FIVE_HTHG","H_STD_FIVE_HTR_A","H_STD_FIVE_HTR_D","H_STD_FIVE_HTR_H","H_STD_FIVE_HY","A_STD_THREE_AC","A_STD_THREE_AF","A_STD_THREE_AR","A_STD_THREE_AS","A_STD_THREE_AST","A_STD_THREE_AY","A_STD_THREE_FTAG","A_STD_THREE_FTHG","A_STD_THREE_FTR_A","A_STD_THREE_FTR_D","A_STD_THREE_FTR_H","A_STD_THREE_HC","A_STD_THREE_HF","A_STD_THREE_HR","A_STD_THREE_HS","A_STD_THREE_HST","A_STD_THREE_HTAG","A_STD_THREE_HTHG","A_STD_THREE_HTR_A","A_STD_THREE_HTR_D","A_STD_THREE_HTR_H","A_STD_THREE_HY","H_STD_THREE_AC","H_STD_THREE_AF","H_STD_THREE_AR","H_STD_THREE_AS","H_STD_THREE_AST","H_STD_THREE_AY","H_STD_THREE_FTAG","H_STD_THREE_FTHG","H_STD_THREE_FTR_A","H_STD_THREE_FTR_D","H_STD_THREE_FTR_H","H_STD_THREE_HC","H_STD_THREE_HF","H_STD_THREE_HR","H_STD_THREE_HS","H_STD_THREE_HST","H_STD_THREE_HTAG","H_STD_THREE_HTHG","H_STD_THREE_HTR_A","H_STD_THREE_HTR_D","H_STD_THREE_HTR_H","H_STD_THREE_HY","INFO_Div"]
#all_features = ['A_MEANS_FIVE_AC','A_MEANS_FIVE_AF','A_MEANS_FIVE_AR','A_MEANS_FIVE_AS','A_MEANS_FIVE_AST','A_MEANS_FIVE_AY','A_MEANS_FIVE_FTAG','A_MEANS_FIVE_FTHG','A_MEANS_FIVE_FTR_A','A_MEANS_FIVE_FTR_D','A_MEANS_FIVE_FTR_H','A_MEANS_FIVE_HC','A_MEANS_FIVE_HF','A_MEANS_FIVE_HR','A_MEANS_FIVE_HS','A_MEANS_FIVE_HST','A_MEANS_FIVE_HTAG','A_MEANS_FIVE_HTHG','A_MEANS_FIVE_HTR_A','A_MEANS_FIVE_HTR_D','A_MEANS_FIVE_HTR_H','A_MEANS_FIVE_HY','H_MEANS_FIVE_AC','H_MEANS_FIVE_AF','H_MEANS_FIVE_AR','H_MEANS_FIVE_AS','H_MEANS_FIVE_AST','H_MEANS_FIVE_AY','H_MEANS_FIVE_FTAG','H_MEANS_FIVE_FTHG','H_MEANS_FIVE_FTR_A','H_MEANS_FIVE_FTR_D','H_MEANS_FIVE_FTR_H','H_MEANS_FIVE_HC','H_MEANS_FIVE_HF','H_MEANS_FIVE_HR','H_MEANS_FIVE_HS','H_MEANS_FIVE_HST','H_MEANS_FIVE_HTAG','H_MEANS_FIVE_HTHG','H_MEANS_FIVE_HTR_A','H_MEANS_FIVE_HTR_D','H_MEANS_FIVE_HTR_H','H_MEANS_FIVE_HY','A_MEANS_THREE_AC','A_MEANS_THREE_AF','A_MEANS_THREE_AR','A_MEANS_THREE_AS','A_MEANS_THREE_AST','A_MEANS_THREE_AY','A_MEANS_THREE_FTAG','A_MEANS_THREE_FTHG','A_MEANS_THREE_FTR_A','A_MEANS_THREE_FTR_D','A_MEANS_THREE_FTR_H','A_MEANS_THREE_HC','A_MEANS_THREE_HF','A_MEANS_THREE_HR','A_MEANS_THREE_HS','A_MEANS_THREE_HST','A_MEANS_THREE_HTAG','A_MEANS_THREE_HTHG','A_MEANS_THREE_HTR_A','A_MEANS_THREE_HTR_D','A_MEANS_THREE_HTR_H','A_MEANS_THREE_HY','H_MEANS_THREE_AC','H_MEANS_THREE_AF','H_MEANS_THREE_AR','H_MEANS_THREE_AS','H_MEANS_THREE_AST','H_MEANS_THREE_AY','H_MEANS_THREE_FTAG','H_MEANS_THREE_FTHG','H_MEANS_THREE_FTR_A','H_MEANS_THREE_FTR_D','H_MEANS_THREE_FTR_H','H_MEANS_THREE_HC','H_MEANS_THREE_HF','H_MEANS_THREE_HR','H_MEANS_THREE_HS','H_MEANS_THREE_HST','H_MEANS_THREE_HTAG','H_MEANS_THREE_HTHG','H_MEANS_THREE_HTR_A','H_MEANS_THREE_HTR_D','H_MEANS_THREE_HTR_H','H_MEANS_THREE_HY','A_STD_FIVE_AC','A_STD_FIVE_AF','A_STD_FIVE_AR','A_STD_FIVE_AS','A_STD_FIVE_AST','A_STD_FIVE_AY','A_STD_FIVE_FTAG','A_STD_FIVE_FTHG','A_STD_FIVE_FTR_A','A_STD_FIVE_FTR_D','A_STD_FIVE_FTR_H','A_STD_FIVE_HC','A_STD_FIVE_HF','A_STD_FIVE_HR','A_STD_FIVE_HS','A_STD_FIVE_HST','A_STD_FIVE_HTAG','A_STD_FIVE_HTHG','A_STD_FIVE_HTR_A','A_STD_FIVE_HTR_D','A_STD_FIVE_HTR_H','A_STD_FIVE_HY','H_STD_FIVE_AC','H_STD_FIVE_AF','H_STD_FIVE_AR','H_STD_FIVE_AS','H_STD_FIVE_AST','H_STD_FIVE_AY','H_STD_FIVE_FTAG','H_STD_FIVE_FTHG','H_STD_FIVE_FTR_A','H_STD_FIVE_FTR_D','H_STD_FIVE_FTR_H','H_STD_FIVE_HC','H_STD_FIVE_HF','H_STD_FIVE_HR','H_STD_FIVE_HS','H_STD_FIVE_HST','H_STD_FIVE_HTAG','H_STD_FIVE_HTHG','H_STD_FIVE_HTR_A','H_STD_FIVE_HTR_D','H_STD_FIVE_HTR_H','H_STD_FIVE_HY','A_STD_THREE_AC','A_STD_THREE_AF','A_STD_THREE_AR','A_STD_THREE_AS','A_STD_THREE_AST','A_STD_THREE_AY','A_STD_THREE_FTAG','A_STD_THREE_FTHG','A_STD_THREE_FTR_A','A_STD_THREE_FTR_D','A_STD_THREE_FTR_H','A_STD_THREE_HC','A_STD_THREE_HF','A_STD_THREE_HR','A_STD_THREE_HS','A_STD_THREE_HST','A_STD_THREE_HTAG','A_STD_THREE_HTHG','A_STD_THREE_HTR_A','A_STD_THREE_HTR_D','A_STD_THREE_HTR_H','A_STD_THREE_HY','H_STD_THREE_AC','H_STD_THREE_AF','H_STD_THREE_AR','H_STD_THREE_AS','H_STD_THREE_AST','H_STD_THREE_AY','H_STD_THREE_FTAG','H_STD_THREE_FTHG','H_STD_THREE_FTR_A','H_STD_THREE_FTR_D','H_STD_THREE_FTR_H','H_STD_THREE_HC','H_STD_THREE_HF','H_STD_THREE_HR','H_STD_THREE_HS','H_STD_THREE_HST','H_STD_THREE_HTAG','H_STD_THREE_HTHG','H_STD_THREE_HTR_A','H_STD_THREE_HTR_D','H_STD_THREE_HTR_H','H_STD_THREE_HY','INFO_Div_D1','INFO_Div_E0','INFO_Div_E1','INFO_Div_E2','INFO_Div_F1','INFO_Div_I1','INFO_Div_SC0','INFO_Div_SP1']
# All features without info_div
all_features = ['A_MEANS_FIVE_AC','A_MEANS_FIVE_AF','A_MEANS_FIVE_AR','A_MEANS_FIVE_AS','A_MEANS_FIVE_AST','A_MEANS_FIVE_AY','A_MEANS_FIVE_FTAG','A_MEANS_FIVE_FTHG','A_MEANS_FIVE_FTR_A','A_MEANS_FIVE_FTR_D','A_MEANS_FIVE_FTR_H','A_MEANS_FIVE_HC','A_MEANS_FIVE_HF','A_MEANS_FIVE_HR','A_MEANS_FIVE_HS','A_MEANS_FIVE_HST','A_MEANS_FIVE_HTAG','A_MEANS_FIVE_HTHG','A_MEANS_FIVE_HTR_A','A_MEANS_FIVE_HTR_D','A_MEANS_FIVE_HTR_H','A_MEANS_FIVE_HY','H_MEANS_FIVE_AC','H_MEANS_FIVE_AF','H_MEANS_FIVE_AR','H_MEANS_FIVE_AS','H_MEANS_FIVE_AST','H_MEANS_FIVE_AY','H_MEANS_FIVE_FTAG','H_MEANS_FIVE_FTHG','H_MEANS_FIVE_FTR_A','H_MEANS_FIVE_FTR_D','H_MEANS_FIVE_FTR_H','H_MEANS_FIVE_HC','H_MEANS_FIVE_HF','H_MEANS_FIVE_HR','H_MEANS_FIVE_HS','H_MEANS_FIVE_HST','H_MEANS_FIVE_HTAG','H_MEANS_FIVE_HTHG','H_MEANS_FIVE_HTR_A','H_MEANS_FIVE_HTR_D','H_MEANS_FIVE_HTR_H','H_MEANS_FIVE_HY','A_MEANS_THREE_AC','A_MEANS_THREE_AF','A_MEANS_THREE_AR','A_MEANS_THREE_AS','A_MEANS_THREE_AST','A_MEANS_THREE_AY','A_MEANS_THREE_FTAG','A_MEANS_THREE_FTHG','A_MEANS_THREE_FTR_A','A_MEANS_THREE_FTR_D','A_MEANS_THREE_FTR_H','A_MEANS_THREE_HC','A_MEANS_THREE_HF','A_MEANS_THREE_HR','A_MEANS_THREE_HS','A_MEANS_THREE_HST','A_MEANS_THREE_HTAG','A_MEANS_THREE_HTHG','A_MEANS_THREE_HTR_A','A_MEANS_THREE_HTR_D','A_MEANS_THREE_HTR_H','A_MEANS_THREE_HY','H_MEANS_THREE_AC','H_MEANS_THREE_AF','H_MEANS_THREE_AR','H_MEANS_THREE_AS','H_MEANS_THREE_AST','H_MEANS_THREE_AY','H_MEANS_THREE_FTAG','H_MEANS_THREE_FTHG','H_MEANS_THREE_FTR_A','H_MEANS_THREE_FTR_D','H_MEANS_THREE_FTR_H','H_MEANS_THREE_HC','H_MEANS_THREE_HF','H_MEANS_THREE_HR','H_MEANS_THREE_HS','H_MEANS_THREE_HST','H_MEANS_THREE_HTAG','H_MEANS_THREE_HTHG','H_MEANS_THREE_HTR_A','H_MEANS_THREE_HTR_D','H_MEANS_THREE_HTR_H','H_MEANS_THREE_HY','A_STD_FIVE_AC','A_STD_FIVE_AF','A_STD_FIVE_AR','A_STD_FIVE_AS','A_STD_FIVE_AST','A_STD_FIVE_AY','A_STD_FIVE_FTAG','A_STD_FIVE_FTHG','A_STD_FIVE_FTR_A','A_STD_FIVE_FTR_D','A_STD_FIVE_FTR_H','A_STD_FIVE_HC','A_STD_FIVE_HF','A_STD_FIVE_HR','A_STD_FIVE_HS','A_STD_FIVE_HST','A_STD_FIVE_HTAG','A_STD_FIVE_HTHG','A_STD_FIVE_HTR_A','A_STD_FIVE_HTR_D','A_STD_FIVE_HTR_H','A_STD_FIVE_HY','H_STD_FIVE_AC','H_STD_FIVE_AF','H_STD_FIVE_AR','H_STD_FIVE_AS','H_STD_FIVE_AST','H_STD_FIVE_AY','H_STD_FIVE_FTAG','H_STD_FIVE_FTHG','H_STD_FIVE_FTR_A','H_STD_FIVE_FTR_D','H_STD_FIVE_FTR_H','H_STD_FIVE_HC','H_STD_FIVE_HF','H_STD_FIVE_HR','H_STD_FIVE_HS','H_STD_FIVE_HST','H_STD_FIVE_HTAG','H_STD_FIVE_HTHG','H_STD_FIVE_HTR_A','H_STD_FIVE_HTR_D','H_STD_FIVE_HTR_H','H_STD_FIVE_HY','A_STD_THREE_AC','A_STD_THREE_AF','A_STD_THREE_AR','A_STD_THREE_AS','A_STD_THREE_AST','A_STD_THREE_AY','A_STD_THREE_FTAG','A_STD_THREE_FTHG','A_STD_THREE_FTR_A','A_STD_THREE_FTR_D','A_STD_THREE_FTR_H','A_STD_THREE_HC','A_STD_THREE_HF','A_STD_THREE_HR','A_STD_THREE_HS','A_STD_THREE_HST','A_STD_THREE_HTAG','A_STD_THREE_HTHG','A_STD_THREE_HTR_A','A_STD_THREE_HTR_D','A_STD_THREE_HTR_H','A_STD_THREE_HY','H_STD_THREE_AC','H_STD_THREE_AF','H_STD_THREE_AR','H_STD_THREE_AS','H_STD_THREE_AST','H_STD_THREE_AY','H_STD_THREE_FTAG','H_STD_THREE_FTHG','H_STD_THREE_FTR_A','H_STD_THREE_FTR_D','H_STD_THREE_FTR_H','H_STD_THREE_HC','H_STD_THREE_HF','H_STD_THREE_HR','H_STD_THREE_HS','H_STD_THREE_HST','H_STD_THREE_HTAG','H_STD_THREE_HTHG','H_STD_THREE_HTR_A','H_STD_THREE_HTR_D','H_STD_THREE_HTR_H','H_STD_THREE_HY']
best_features_DNN_adam_38 = ['A_MEANS_FIVE_AC', 'A_MEANS_FIVE_AS', 'A_MEANS_FIVE_AST','A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_FTHG', 'A_MEANS_FIVE_FTR_H','A_MEANS_FIVE_HC', 'A_MEANS_FIVE_HF', 'A_MEANS_FIVE_HS','A_MEANS_FIVE_HST', 'A_MEANS_FIVE_HTR_A', 'A_MEANS_FIVE_HY','H_MEANS_FIVE_AC', 'H_MEANS_FIVE_AS', 'H_MEANS_FIVE_AST','H_MEANS_FIVE_AY', 'H_MEANS_FIVE_FTAG', 'H_MEANS_FIVE_FTHG','H_MEANS_FIVE_FTR_A', 'H_MEANS_FIVE_FTR_H', 'H_MEANS_FIVE_HC','H_MEANS_FIVE_HS', 'H_MEANS_FIVE_HST', 'H_MEANS_FIVE_HTR_H','A_MEANS_THREE_AC', 'A_MEANS_THREE_AS', 'A_MEANS_THREE_FTHG','A_MEANS_THREE_HF', 'A_MEANS_THREE_HS', 'H_MEANS_THREE_AS','H_MEANS_THREE_HC', 'A_STD_FIVE_AST', 'A_STD_FIVE_HF','H_STD_FIVE_AF', 'H_STD_FIVE_AS', 'H_STD_FIVE_HC', 'H_STD_FIVE_HF','H_STD_FIVE_HST']
best_features_DNN_SGD_56 = ['A_MEANS_FIVE_AC', 'A_MEANS_FIVE_AS', 'A_MEANS_FIVE_AST','A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_FTHG', 'A_MEANS_FIVE_FTR_A','A_MEANS_FIVE_FTR_H', 'A_MEANS_FIVE_HC', 'A_MEANS_FIVE_HF','A_MEANS_FIVE_HS', 'A_MEANS_FIVE_HST', 'A_MEANS_FIVE_HTHG','A_MEANS_FIVE_HTR_A', 'A_MEANS_FIVE_HY', 'H_MEANS_FIVE_AC','H_MEANS_FIVE_AS', 'H_MEANS_FIVE_AST', 'H_MEANS_FIVE_AY','H_MEANS_FIVE_FTAG', 'H_MEANS_FIVE_FTHG', 'H_MEANS_FIVE_FTR_A','H_MEANS_FIVE_FTR_H', 'H_MEANS_FIVE_HC', 'H_MEANS_FIVE_HF','H_MEANS_FIVE_HS', 'H_MEANS_FIVE_HST', 'H_MEANS_FIVE_HTAG','H_MEANS_FIVE_HTR_H', 'A_MEANS_THREE_AC', 'A_MEANS_THREE_AS','A_MEANS_THREE_FTHG', 'A_MEANS_THREE_FTR_A', 'A_MEANS_THREE_HC','A_MEANS_THREE_HF', 'A_MEANS_THREE_HS', 'A_MEANS_THREE_HST','H_MEANS_THREE_AC', 'H_MEANS_THREE_AS', 'H_MEANS_THREE_FTHG','H_MEANS_THREE_HC', 'H_MEANS_THREE_HST', 'H_MEANS_THREE_HTR_H','A_STD_FIVE_AF', 'A_STD_FIVE_AST', 'A_STD_FIVE_HC', 'A_STD_FIVE_HF','A_STD_FIVE_HS', 'H_STD_FIVE_AF', 'H_STD_FIVE_AS', 'H_STD_FIVE_AST','H_STD_FIVE_HC', 'H_STD_FIVE_HF', 'H_STD_FIVE_HST','H_STD_FIVE_HTHG', 'H_STD_THREE_AS', 'H_STD_THREE_HST']
best_features_XGBoost_60 = ['A_MEANS_FIVE_AC', 'A_MEANS_FIVE_AS', 'A_MEANS_FIVE_AST','A_MEANS_FIVE_AY', 'A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_FTHG','A_MEANS_FIVE_FTR_A', 'A_MEANS_FIVE_FTR_H', 'A_MEANS_FIVE_HC','A_MEANS_FIVE_HF', 'A_MEANS_FIVE_HS', 'A_MEANS_FIVE_HST','A_MEANS_FIVE_HTHG', 'A_MEANS_FIVE_HTR_A', 'A_MEANS_FIVE_HY','H_MEANS_FIVE_AC', 'H_MEANS_FIVE_AS', 'H_MEANS_FIVE_AST','H_MEANS_FIVE_AY', 'H_MEANS_FIVE_FTAG', 'H_MEANS_FIVE_FTHG','H_MEANS_FIVE_FTR_A', 'H_MEANS_FIVE_FTR_H', 'H_MEANS_FIVE_HC','H_MEANS_FIVE_HF', 'H_MEANS_FIVE_HS', 'H_MEANS_FIVE_HST','H_MEANS_FIVE_HTAG', 'H_MEANS_FIVE_HTR_A', 'H_MEANS_FIVE_HTR_H','A_MEANS_THREE_AC', 'A_MEANS_THREE_AS', 'A_MEANS_THREE_FTHG','A_MEANS_THREE_FTR_A', 'A_MEANS_THREE_HC', 'A_MEANS_THREE_HF','A_MEANS_THREE_HS', 'A_MEANS_THREE_HST', 'H_MEANS_THREE_AC','H_MEANS_THREE_AS', 'H_MEANS_THREE_AST', 'H_MEANS_THREE_FTHG','H_MEANS_THREE_HC', 'H_MEANS_THREE_HST', 'H_MEANS_THREE_HTR_H','A_STD_FIVE_AF', 'A_STD_FIVE_AS', 'A_STD_FIVE_AST', 'A_STD_FIVE_HC','A_STD_FIVE_HF', 'A_STD_FIVE_HS', 'H_STD_FIVE_AF', 'H_STD_FIVE_AS','H_STD_FIVE_AST', 'H_STD_FIVE_HC', 'H_STD_FIVE_HF','H_STD_FIVE_HST', 'H_STD_FIVE_HTHG', 'H_STD_THREE_AS','H_STD_THREE_HST']
best_features_SVM = ['A_MEANS_FIVE_AC', 'A_MEANS_FIVE_AS', 'A_MEANS_FIVE_AST','A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_FTHG', 'A_MEANS_FIVE_FTR_H','A_MEANS_FIVE_HC', 'A_MEANS_FIVE_HS', 'A_MEANS_FIVE_HST','A_MEANS_FIVE_HTR_A', 'H_MEANS_FIVE_AC', 'H_MEANS_FIVE_AS','H_MEANS_FIVE_AST', 'H_MEANS_FIVE_AY', 'H_MEANS_FIVE_FTAG','H_MEANS_FIVE_FTHG', 'H_MEANS_FIVE_FTR_A', 'H_MEANS_FIVE_FTR_H','H_MEANS_FIVE_HC', 'H_MEANS_FIVE_HS', 'H_MEANS_FIVE_HST','H_MEANS_FIVE_HTR_H', 'A_MEANS_THREE_AC', 'A_MEANS_THREE_AS','A_MEANS_THREE_FTHG', 'A_MEANS_THREE_HS', 'H_MEANS_THREE_AS','A_STD_FIVE_HF', 'H_STD_FIVE_HC', 'H_STD_FIVE_HST']
best_features_RF = ['A_MEANS_FIVE_AC', 'A_MEANS_FIVE_AS', 'A_MEANS_FIVE_AST','A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_FTHG', 'A_MEANS_FIVE_FTR_A','A_MEANS_FIVE_FTR_H', 'A_MEANS_FIVE_HC', 'A_MEANS_FIVE_HF','A_MEANS_FIVE_HS', 'A_MEANS_FIVE_HST', 'A_MEANS_FIVE_HTHG','A_MEANS_FIVE_HTR_A', 'A_MEANS_FIVE_HY', 'H_MEANS_FIVE_AC','H_MEANS_FIVE_AS', 'H_MEANS_FIVE_AST', 'H_MEANS_FIVE_AY','H_MEANS_FIVE_FTAG', 'H_MEANS_FIVE_FTHG', 'H_MEANS_FIVE_FTR_A','H_MEANS_FIVE_FTR_H', 'H_MEANS_FIVE_HC', 'H_MEANS_FIVE_HF','H_MEANS_FIVE_HS', 'H_MEANS_FIVE_HST', 'H_MEANS_FIVE_HTAG','H_MEANS_FIVE_HTR_H', 'A_MEANS_THREE_AC', 'A_MEANS_THREE_AS','A_MEANS_THREE_FTHG', 'A_MEANS_THREE_HC', 'A_MEANS_THREE_HF','A_MEANS_THREE_HS', 'H_MEANS_THREE_AC', 'H_MEANS_THREE_AS','H_MEANS_THREE_FTHG', 'H_MEANS_THREE_HC', 'H_MEANS_THREE_HST','A_STD_FIVE_AF', 'A_STD_FIVE_AST', 'A_STD_FIVE_HC', 'A_STD_FIVE_HF','A_STD_FIVE_HS', 'H_STD_FIVE_AF', 'H_STD_FIVE_AS', 'H_STD_FIVE_AST','H_STD_FIVE_HC', 'H_STD_FIVE_HF', 'H_STD_FIVE_HST','H_STD_FIVE_HTHG', 'H_STD_THREE_AS', 'H_STD_THREE_HST']
best_features_NB = ['A_MEANS_FIVE_AC', 'A_MEANS_FIVE_AS', 'A_MEANS_FIVE_AST','A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_FTHG', 'A_MEANS_FIVE_FTR_H','A_MEANS_FIVE_HC', 'A_MEANS_FIVE_HS', 'A_MEANS_FIVE_HST','A_MEANS_FIVE_HTR_A', 'H_MEANS_FIVE_AC', 'H_MEANS_FIVE_AS','H_MEANS_FIVE_AST', 'H_MEANS_FIVE_AY', 'H_MEANS_FIVE_FTAG','H_MEANS_FIVE_FTHG', 'H_MEANS_FIVE_FTR_A', 'H_MEANS_FIVE_FTR_H','H_MEANS_FIVE_HC', 'H_MEANS_FIVE_HS', 'H_MEANS_FIVE_HST','H_MEANS_FIVE_HTR_H', 'A_MEANS_THREE_AC', 'A_MEANS_THREE_AS','A_MEANS_THREE_FTHG', 'A_MEANS_THREE_HS', 'H_MEANS_THREE_AS','A_STD_FIVE_HF', 'H_STD_FIVE_HC', 'H_STD_FIVE_HST']
best_features_MLP = ['A_MEANS_FIVE_AC', 'A_MEANS_FIVE_AS', 'A_MEANS_FIVE_AST','A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_FTHG', 'A_MEANS_FIVE_FTR_H','A_MEANS_FIVE_HC', 'A_MEANS_FIVE_HS', 'A_MEANS_FIVE_HST','A_MEANS_FIVE_HTR_A', 'H_MEANS_FIVE_AC', 'H_MEANS_FIVE_AS','H_MEANS_FIVE_AST', 'H_MEANS_FIVE_AY', 'H_MEANS_FIVE_FTAG','H_MEANS_FIVE_FTHG', 'H_MEANS_FIVE_FTR_A', 'H_MEANS_FIVE_FTR_H','H_MEANS_FIVE_HC', 'H_MEANS_FIVE_HS', 'H_MEANS_FIVE_HST','H_MEANS_FIVE_HTR_H', 'A_MEANS_THREE_AC', 'A_MEANS_THREE_AS','A_MEANS_THREE_FTHG', 'A_MEANS_THREE_HS', 'H_MEANS_THREE_AS','A_STD_FIVE_HF', 'H_STD_FIVE_HC', 'H_STD_FIVE_HST']
best_features_LDA = ['A_MEANS_FIVE_AS', 'A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_HC','A_MEANS_FIVE_HS', 'H_MEANS_FIVE_AS', 'H_MEANS_FIVE_AST','H_MEANS_FIVE_FTHG', 'H_MEANS_FIVE_HC', 'H_MEANS_FIVE_HS','H_MEANS_FIVE_HST']
best_features_KNN = ['A_MEANS_FIVE_AC', 'A_MEANS_FIVE_AS', 'A_MEANS_FIVE_AST','A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_HC', 'A_MEANS_FIVE_HS','A_MEANS_FIVE_HST', 'A_MEANS_FIVE_HTR_A', 'H_MEANS_FIVE_AC','H_MEANS_FIVE_AS', 'H_MEANS_FIVE_AST', 'H_MEANS_FIVE_AY','H_MEANS_FIVE_FTAG', 'H_MEANS_FIVE_FTHG', 'H_MEANS_FIVE_FTR_A','H_MEANS_FIVE_HC', 'H_MEANS_FIVE_HS', 'H_MEANS_FIVE_HST','A_MEANS_THREE_AS', 'A_MEANS_THREE_HS', 'A_STD_FIVE_HF']
best_features_ExraTree = ['A_MEANS_FIVE_AC', 'A_MEANS_FIVE_AS', 'A_MEANS_FIVE_AST','A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_FTHG', 'A_MEANS_FIVE_FTR_A','A_MEANS_FIVE_FTR_H', 'A_MEANS_FIVE_HC', 'A_MEANS_FIVE_HF','A_MEANS_FIVE_HS', 'A_MEANS_FIVE_HST', 'A_MEANS_FIVE_HTHG','A_MEANS_FIVE_HTR_A', 'A_MEANS_FIVE_HY', 'H_MEANS_FIVE_AC','H_MEANS_FIVE_AS', 'H_MEANS_FIVE_AST', 'H_MEANS_FIVE_AY','H_MEANS_FIVE_FTAG', 'H_MEANS_FIVE_FTHG', 'H_MEANS_FIVE_FTR_A','H_MEANS_FIVE_FTR_H', 'H_MEANS_FIVE_HC', 'H_MEANS_FIVE_HF','H_MEANS_FIVE_HS', 'H_MEANS_FIVE_HST', 'H_MEANS_FIVE_HTAG','H_MEANS_FIVE_HTR_H', 'A_MEANS_THREE_AC', 'A_MEANS_THREE_AS','A_MEANS_THREE_FTHG', 'A_MEANS_THREE_FTR_A', 'A_MEANS_THREE_HC','A_MEANS_THREE_HF', 'A_MEANS_THREE_HS', 'A_MEANS_THREE_HST','H_MEANS_THREE_AC', 'H_MEANS_THREE_AS', 'H_MEANS_THREE_FTHG','H_MEANS_THREE_HC', 'H_MEANS_THREE_HST', 'H_MEANS_THREE_HTR_H','A_STD_FIVE_AF', 'A_STD_FIVE_AST', 'A_STD_FIVE_HC', 'A_STD_FIVE_HF','A_STD_FIVE_HS', 'H_STD_FIVE_AF', 'H_STD_FIVE_AS', 'H_STD_FIVE_AST','H_STD_FIVE_HC', 'H_STD_FIVE_HF', 'H_STD_FIVE_HST','H_STD_FIVE_HTHG', 'H_STD_THREE_AS', 'H_STD_THREE_HST']
best_features_ElasticNEt = ['A_MEANS_FIVE_AC', 'A_MEANS_FIVE_AS', 'A_MEANS_FIVE_AST','A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_FTR_H', 'A_MEANS_FIVE_HC','A_MEANS_FIVE_HS', 'A_MEANS_FIVE_HST', 'A_MEANS_FIVE_HTR_A','H_MEANS_FIVE_AC', 'H_MEANS_FIVE_AS', 'H_MEANS_FIVE_AST','H_MEANS_FIVE_AY', 'H_MEANS_FIVE_FTAG', 'H_MEANS_FIVE_FTHG','H_MEANS_FIVE_FTR_A', 'H_MEANS_FIVE_HC', 'H_MEANS_FIVE_HS','H_MEANS_FIVE_HST', 'A_MEANS_THREE_AS', 'A_MEANS_THREE_FTHG','A_MEANS_THREE_HS', 'A_STD_FIVE_HF', 'H_STD_FIVE_HST']
best_features_bagging = ['A_MEANS_FIVE_AS', 'A_MEANS_FIVE_AST', 'A_MEANS_FIVE_FTAG','A_MEANS_FIVE_HC', 'A_MEANS_FIVE_HS', 'A_MEANS_FIVE_HST','H_MEANS_FIVE_AC', 'H_MEANS_FIVE_AS', 'H_MEANS_FIVE_AST','H_MEANS_FIVE_AY', 'H_MEANS_FIVE_FTHG', 'H_MEANS_FIVE_FTR_A','H_MEANS_FIVE_HC', 'H_MEANS_FIVE_HS', 'H_MEANS_FIVE_HST','A_MEANS_THREE_HS']
best_features_AdaBoost = ['A_MEANS_FIVE_AC', 'A_MEANS_FIVE_AS', 'A_MEANS_FIVE_AST','A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_FTHG', 'A_MEANS_FIVE_FTR_H','A_MEANS_FIVE_HC', 'A_MEANS_FIVE_HS', 'A_MEANS_FIVE_HST','A_MEANS_FIVE_HTR_A', 'A_MEANS_FIVE_HY', 'H_MEANS_FIVE_AC','H_MEANS_FIVE_AS', 'H_MEANS_FIVE_AST', 'H_MEANS_FIVE_AY','H_MEANS_FIVE_FTAG', 'H_MEANS_FIVE_FTHG', 'H_MEANS_FIVE_FTR_A','H_MEANS_FIVE_FTR_H', 'H_MEANS_FIVE_HC', 'H_MEANS_FIVE_HS','H_MEANS_FIVE_HST', 'H_MEANS_FIVE_HTR_H', 'A_MEANS_THREE_AC','A_MEANS_THREE_AS', 'A_MEANS_THREE_FTHG', 'A_MEANS_THREE_HS','H_MEANS_THREE_AS', 'H_MEANS_THREE_HC', 'A_STD_FIVE_AST','A_STD_FIVE_HF', 'H_STD_FIVE_AF', 'H_STD_FIVE_HC','H_STD_FIVE_HST']
best_features_LR = ['A_MEANS_FIVE_AC', 'A_MEANS_FIVE_AS', 'A_MEANS_FIVE_AST','A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_FTHG', 'A_MEANS_FIVE_FTR_H','A_MEANS_FIVE_HC', 'A_MEANS_FIVE_HF', 'A_MEANS_FIVE_HS','A_MEANS_FIVE_HST', 'A_MEANS_FIVE_HTR_A', 'A_MEANS_FIVE_HY','H_MEANS_FIVE_AC', 'H_MEANS_FIVE_AS', 'H_MEANS_FIVE_AST','H_MEANS_FIVE_AY', 'H_MEANS_FIVE_FTAG', 'H_MEANS_FIVE_FTHG','H_MEANS_FIVE_FTR_A', 'H_MEANS_FIVE_FTR_H', 'H_MEANS_FIVE_HC','H_MEANS_FIVE_HF', 'H_MEANS_FIVE_HS', 'H_MEANS_FIVE_HST','H_MEANS_FIVE_HTAG', 'H_MEANS_FIVE_HTR_H', 'A_MEANS_THREE_AC','A_MEANS_THREE_AS', 'A_MEANS_THREE_FTHG', 'A_MEANS_THREE_HF','A_MEANS_THREE_HS', 'H_MEANS_THREE_AS', 'H_MEANS_THREE_HC','A_STD_FIVE_AST', 'A_STD_FIVE_HC', 'A_STD_FIVE_HF', 'A_STD_FIVE_HS','H_STD_FIVE_AF', 'H_STD_FIVE_AS', 'H_STD_FIVE_AST', 'H_STD_FIVE_HC','H_STD_FIVE_HF', 'H_STD_FIVE_HST']
best_features_GB = ['A_MEANS_FIVE_AC', 'A_MEANS_FIVE_AS', 'A_MEANS_FIVE_AST','A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_FTHG', 'A_MEANS_FIVE_FTR_A','A_MEANS_FIVE_FTR_H', 'A_MEANS_FIVE_HC', 'A_MEANS_FIVE_HF','A_MEANS_FIVE_HS', 'A_MEANS_FIVE_HST', 'A_MEANS_FIVE_HTHG','A_MEANS_FIVE_HTR_A', 'A_MEANS_FIVE_HY', 'H_MEANS_FIVE_AC','H_MEANS_FIVE_AS', 'H_MEANS_FIVE_AST', 'H_MEANS_FIVE_AY','H_MEANS_FIVE_FTAG', 'H_MEANS_FIVE_FTHG', 'H_MEANS_FIVE_FTR_A','H_MEANS_FIVE_FTR_H', 'H_MEANS_FIVE_HC', 'H_MEANS_FIVE_HF','H_MEANS_FIVE_HS', 'H_MEANS_FIVE_HST', 'H_MEANS_FIVE_HTAG','H_MEANS_FIVE_HTR_H', 'A_MEANS_THREE_AC', 'A_MEANS_THREE_AS','A_MEANS_THREE_FTHG', 'A_MEANS_THREE_HC', 'A_MEANS_THREE_HF','A_MEANS_THREE_HS', 'H_MEANS_THREE_AC', 'H_MEANS_THREE_AS','H_MEANS_THREE_FTHG', 'H_MEANS_THREE_HC', 'H_MEANS_THREE_HST','A_STD_FIVE_AF', 'A_STD_FIVE_AST', 'A_STD_FIVE_HC', 'A_STD_FIVE_HF','A_STD_FIVE_HS', 'H_STD_FIVE_AF', 'H_STD_FIVE_AS', 'H_STD_FIVE_AST','H_STD_FIVE_HC', 'H_STD_FIVE_HF', 'H_STD_FIVE_HST','H_STD_FIVE_HTHG', 'H_STD_THREE_AS', 'H_STD_THREE_HST']

In [7]:
# Construct layer 1
# [Model_name, Preprocessing_needed, calibration, model_with_hyperparameter, feature_list]
layer_1 = [
    ['DNN_adam_all', True, 'sigmoid', KerasClassifier(build_fn=create_DNN_model_adam_all,verbose=0,batch_size=1500,epochs=3), all_features],
    ['DNN_adam_bf', True, 'isotonic', KerasClassifier(build_fn=create_DNN_model_adam_38,verbose=0,batch_size=1500,epochs=3), best_features_DNN_adam_38],
    ['DNN_sgd_all', True, 'sigmoid', KerasClassifier(build_fn=create_DNN_model_sgd_all,verbose=0,batch_size=80,epochs=5), all_features],
    ['DNN_sgd_bf', True, 'isotonic', KerasClassifier(build_fn=create_DNN_model_sgd_56,verbose=0,batch_size=80,epochs=5), best_features_DNN_SGD_56],
    ['XGBoost_all', False, 'sigmoid', XGBClassifier(learning_rate =0.01,n_estimators=572,max_depth=3,min_child_weight=5,gamma=0,subsample=0.9,colsample_bytree=0.9,objective= 'binary:logistic',nthread=4,scale_pos_weight=1,seed=15), all_features],
    ['XGBoost_bf', False, 'no', XGBClassifier(learning_rate =0.01,n_estimators=716,max_depth=3,min_child_weight=5,gamma=0,subsample=0.9,colsample_bytree=0.9,objective= 'binary:logistic',nthread=4,scale_pos_weight=1,seed=15), best_features_XGBoost_60],
    ['SVM', True, 'sigmoid', SVC(random_state=0,C=0.02,kernel='linear',gamma=1,probability=True), best_features_SVM],
    ['RF_all', False, 'isotonic', RandomForestClassifier(random_state=0, n_jobs=-1,criterion='entropy', max_depth=None, max_features='sqrt', min_samples_leaf=1, min_samples_split=2, n_estimators=360), all_features],
    ['RF_bf', False, 'sigmoid', RandomForestClassifier(random_state=0, n_jobs=-1,criterion='entropy', max_depth=None, max_features='sqrt', min_samples_leaf=1, min_samples_split=2, n_estimators=360), best_features_RF],
    ['NB', True, 'isotonic', GaussianNB(), best_features_NB],
    ['MLP', True, 'no', MLPClassifier(random_state=0,activation='identity', alpha=3, hidden_layer_sizes=(10,), max_iter=200, solver= 'adam'), best_features_MLP],
    ['LDA', True, 'no', LinearDiscriminantAnalysis(), best_features_LDA],
    #['KNN', True, 'sigmoid', KNeighborsClassifier(n_jobs=-1,n_neighbors=200, p=2, weights='uniform'), best_features_KNN],
    ['KNN', True, 'sigmoid', KNeighborsClassifier(n_jobs=-1,n_neighbors=50, p=2, weights='uniform'), best_features_KNN],
    ['ExraTree', False, 'isotonic', ExtraTreesClassifier(random_state=0,n_jobs=-1,criterion='entropy',max_depth=30,max_features=None,min_samples_leaf=1,min_samples_split=16,n_estimators=350), best_features_ExraTree],
    ['ElasticNEt', True, 'isotonic', SGDClassifier(random_state=0,alpha=0.0024, l1_ratio=0.64, loss='log', penalty='elasticnet'), best_features_ElasticNEt],
    ['Bagging_max_sample', False, 'sigmoid', BaggingClassifier(random_state=0, n_jobs=-1,max_features=0.7, max_samples=1.0, n_estimators=100), all_features],
    ['Bagging_min_sample_all', False, 'isotonic', BaggingClassifier(random_state=0, n_jobs=-1,max_features=0.5, max_samples=0.1, n_estimators=400), all_features],
    ['Bagging_min_sample_bf', False, 'sigmoid', BaggingClassifier(random_state=0, n_jobs=-1,max_features=0.5, max_samples=0.1, n_estimators=400), best_features_bagging],
    ['AdaBoost', False, 'sigmoid', AdaBoostClassifier(random_state=0,n_estimators=30), best_features_AdaBoost],
    ['GB', False, 'isotonic', GradientBoostingClassifier(random_state=0,criterion='friedman_mse',loss='deviance',max_depth=20,max_features='log2',min_samples_leaf=3,min_samples_split=10,n_estimators=150), best_features_GB],
    ['LR', True, 'no', LogisticRegression(random_state=0,n_jobs=-1,solver='liblinear',C=0.008,penalty='l1'), best_features_LR]
]
# For testing
#layer_1 = [
#    ['AdaBoost', False, 'sigmoid', AdaBoostClassifier(random_state=0,n_estimators=30), best_features_AdaBoost],
#    ['NB', True, 'isotonic', GaussianNB(), best_features_NB],
#    ['LR', True, 'no', LogisticRegression(random_state=0,n_jobs=-1,solver='liblinear',C=0.008,penalty='l1'), best_features_LR]
#]

In [8]:
# Define all different features dataset for layer 2
#to_remove = ['A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_HF', 'A_MEANS_FIVE_HS','A_MEANS_FIVE_HTR_D', 'H_MEANS_FIVE_AF', 'H_MEANS_FIVE_AY','H_MEANS_FIVE_HF', 'A_MEANS_THREE_AF', 'A_MEANS_THREE_HF','H_MEANS_THREE_AF', 'A_STD_FIVE_HF', 'H_STD_FIVE_AS','H_STD_FIVE_AST', 'H_STD_FIVE_FTHG', 'H_STD_FIVE_HTAG','H_STD_FIVE_HY', 'NB', 'AdaBoost']
best_feature_L2_ET = ['A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_HS', 'H_MEANS_FIVE_AF','H_MEANS_FIVE_HF', 'A_MEANS_THREE_AF', 'A_MEANS_THREE_HF','H_MEANS_THREE_AF', 'A_STD_FIVE_HF', 'H_STD_FIVE_AS','H_STD_FIVE_AST', 'H_STD_FIVE_HY', 'DNN_adam_all', 'DNN_adam_bf','DNN_sgd_bf', 'XGBoost_all', 'XGBoost_bf', 'SVM', 'RF_all','RF_bf', 'NB', 'MLP', 'LDA', 'KNN', 'ElasticNEt','Bagging_max_sample', 'Bagging_min_sample_bf', 'AdaBoost', 'GB']
best_features_L2_NB = ['A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_HF', 'A_MEANS_FIVE_HS','A_MEANS_FIVE_HTR_D', 'H_MEANS_FIVE_AF', 'H_MEANS_FIVE_AY','H_MEANS_FIVE_HF', 'A_MEANS_THREE_AF', 'A_MEANS_THREE_HF','H_MEANS_THREE_AF', 'A_STD_FIVE_HF', 'H_STD_FIVE_AS','H_STD_FIVE_AST', 'H_STD_FIVE_FTHG', 'H_STD_FIVE_HTAG','H_STD_FIVE_HY', 'DNN_adam_all', 'DNN_adam_bf', 'DNN_sgd_all','DNN_sgd_bf', 'XGBoost_all', 'XGBoost_bf', 'SVM', 'RF_all','RF_bf', 'NB', 'MLP', 'LDA', 'KNN', 'ExraTree', 'ElasticNEt','Bagging_max_sample', 'Bagging_min_sample_all','Bagging_min_sample_bf', 'AdaBoost', 'GB']
best_features_L2_MLP = ['DNN_sgd_bf', 'RF_all', 'RF_bf', 'NB', 'MLP', 'LDA', 'KNN','Bagging_min_sample_bf']
best_features_L2_LR = ['A_MEANS_FIVE_AC','A_MEANS_FIVE_AF','A_MEANS_FIVE_AS','A_MEANS_FIVE_AST','A_MEANS_FIVE_AY','A_MEANS_FIVE_FTAG','A_MEANS_FIVE_FTR_H','A_MEANS_FIVE_HC','A_MEANS_FIVE_HF','A_MEANS_FIVE_HS','A_MEANS_FIVE_HTR_D','A_MEANS_FIVE_HTR_H','A_MEANS_FIVE_HY','H_MEANS_FIVE_AC','H_MEANS_FIVE_AF','H_MEANS_FIVE_AS','H_MEANS_FIVE_AST','H_MEANS_FIVE_AY','H_MEANS_FIVE_FTAG','H_MEANS_FIVE_FTHG','H_MEANS_FIVE_FTR_A','H_MEANS_FIVE_HC','H_MEANS_FIVE_HF','H_MEANS_FIVE_HR','H_MEANS_FIVE_HST','H_MEANS_FIVE_HTAG','H_MEANS_FIVE_HTR_H','H_MEANS_FIVE_HY','A_MEANS_THREE_AC','A_MEANS_THREE_AF','A_MEANS_THREE_AS','A_MEANS_THREE_FTHG','A_MEANS_THREE_HC','A_MEANS_THREE_HF','A_MEANS_THREE_HS','A_MEANS_THREE_HST','A_MEANS_THREE_HTR_D','A_MEANS_THREE_HY','H_MEANS_THREE_AC','H_MEANS_THREE_AF','H_MEANS_THREE_FTHG','H_MEANS_THREE_HF','H_MEANS_THREE_HS','H_MEANS_THREE_HST','H_MEANS_THREE_HY','A_STD_FIVE_AC','A_STD_FIVE_AF','A_STD_FIVE_AST','A_STD_FIVE_AY','A_STD_FIVE_FTAG','A_STD_FIVE_FTHG','A_STD_FIVE_FTR_D','A_STD_FIVE_FTR_H','A_STD_FIVE_HC','A_STD_FIVE_HF','A_STD_FIVE_HS','A_STD_FIVE_HST','A_STD_FIVE_HY','H_STD_FIVE_AC','H_STD_FIVE_AF','H_STD_FIVE_AS','H_STD_FIVE_AST','H_STD_FIVE_AY','H_STD_FIVE_FTAG','H_STD_FIVE_FTHG','H_STD_FIVE_HF','H_STD_FIVE_HS','H_STD_FIVE_HST','H_STD_FIVE_HTAG','H_STD_FIVE_HTHG','H_STD_FIVE_HY','A_STD_THREE_AF','A_STD_THREE_AS','A_STD_THREE_AST','A_STD_THREE_FTHG','A_STD_THREE_HC','A_STD_THREE_HF','A_STD_THREE_HS','A_STD_THREE_HST','H_STD_THREE_AF','H_STD_THREE_AS','H_STD_THREE_AST','H_STD_THREE_FTAG','H_STD_THREE_FTHG','H_STD_THREE_HF','H_STD_THREE_HS','H_STD_THREE_HST','H_STD_THREE_HTHG','H_STD_THREE_HY','DNN_adam_all','DNN_adam_bf','DNN_sgd_all','DNN_sgd_bf','XGBoost_all','XGBoost_bf','SVM','RF_all','RF_bf','NB','MLP','LDA','KNN','ExraTree','ElasticNEt','Bagging_max_sample','Bagging_min_sample_all','Bagging_min_sample_bf','AdaBoost','GB','LR']

# Define all different features dataset
all_feature_L2 = ['A_MEANS_FIVE_AC','A_MEANS_FIVE_AF','A_MEANS_FIVE_AR','A_MEANS_FIVE_AS','A_MEANS_FIVE_AST','A_MEANS_FIVE_AY','A_MEANS_FIVE_FTAG','A_MEANS_FIVE_FTHG','A_MEANS_FIVE_FTR_A','A_MEANS_FIVE_FTR_D','A_MEANS_FIVE_FTR_H','A_MEANS_FIVE_HC','A_MEANS_FIVE_HF','A_MEANS_FIVE_HR','A_MEANS_FIVE_HS','A_MEANS_FIVE_HST','A_MEANS_FIVE_HTAG','A_MEANS_FIVE_HTHG','A_MEANS_FIVE_HTR_A','A_MEANS_FIVE_HTR_D','A_MEANS_FIVE_HTR_H','A_MEANS_FIVE_HY','H_MEANS_FIVE_AC','H_MEANS_FIVE_AF','H_MEANS_FIVE_AR','H_MEANS_FIVE_AS','H_MEANS_FIVE_AST','H_MEANS_FIVE_AY','H_MEANS_FIVE_FTAG','H_MEANS_FIVE_FTHG','H_MEANS_FIVE_FTR_A','H_MEANS_FIVE_FTR_D','H_MEANS_FIVE_FTR_H','H_MEANS_FIVE_HC','H_MEANS_FIVE_HF','H_MEANS_FIVE_HR','H_MEANS_FIVE_HS','H_MEANS_FIVE_HST','H_MEANS_FIVE_HTAG','H_MEANS_FIVE_HTHG','H_MEANS_FIVE_HTR_A','H_MEANS_FIVE_HTR_D','H_MEANS_FIVE_HTR_H','H_MEANS_FIVE_HY','A_MEANS_THREE_AC','A_MEANS_THREE_AF','A_MEANS_THREE_AR','A_MEANS_THREE_AS','A_MEANS_THREE_AST','A_MEANS_THREE_AY','A_MEANS_THREE_FTAG','A_MEANS_THREE_FTHG','A_MEANS_THREE_FTR_A','A_MEANS_THREE_FTR_D','A_MEANS_THREE_FTR_H','A_MEANS_THREE_HC','A_MEANS_THREE_HF','A_MEANS_THREE_HR','A_MEANS_THREE_HS','A_MEANS_THREE_HST','A_MEANS_THREE_HTAG','A_MEANS_THREE_HTHG','A_MEANS_THREE_HTR_A','A_MEANS_THREE_HTR_D','A_MEANS_THREE_HTR_H','A_MEANS_THREE_HY','H_MEANS_THREE_AC','H_MEANS_THREE_AF','H_MEANS_THREE_AR','H_MEANS_THREE_AS','H_MEANS_THREE_AST','H_MEANS_THREE_AY','H_MEANS_THREE_FTAG','H_MEANS_THREE_FTHG','H_MEANS_THREE_FTR_A','H_MEANS_THREE_FTR_D','H_MEANS_THREE_FTR_H','H_MEANS_THREE_HC','H_MEANS_THREE_HF','H_MEANS_THREE_HR','H_MEANS_THREE_HS','H_MEANS_THREE_HST','H_MEANS_THREE_HTAG','H_MEANS_THREE_HTHG','H_MEANS_THREE_HTR_A','H_MEANS_THREE_HTR_D','H_MEANS_THREE_HTR_H','H_MEANS_THREE_HY','A_STD_FIVE_AC','A_STD_FIVE_AF','A_STD_FIVE_AR','A_STD_FIVE_AS','A_STD_FIVE_AST','A_STD_FIVE_AY','A_STD_FIVE_FTAG','A_STD_FIVE_FTHG','A_STD_FIVE_FTR_A','A_STD_FIVE_FTR_D','A_STD_FIVE_FTR_H','A_STD_FIVE_HC','A_STD_FIVE_HF','A_STD_FIVE_HR','A_STD_FIVE_HS','A_STD_FIVE_HST','A_STD_FIVE_HTAG','A_STD_FIVE_HTHG','A_STD_FIVE_HTR_A','A_STD_FIVE_HTR_D','A_STD_FIVE_HTR_H','A_STD_FIVE_HY','H_STD_FIVE_AC','H_STD_FIVE_AF','H_STD_FIVE_AR','H_STD_FIVE_AS','H_STD_FIVE_AST','H_STD_FIVE_AY','H_STD_FIVE_FTAG','H_STD_FIVE_FTHG','H_STD_FIVE_FTR_A','H_STD_FIVE_FTR_D','H_STD_FIVE_FTR_H','H_STD_FIVE_HC','H_STD_FIVE_HF','H_STD_FIVE_HR','H_STD_FIVE_HS','H_STD_FIVE_HST','H_STD_FIVE_HTAG','H_STD_FIVE_HTHG','H_STD_FIVE_HTR_A','H_STD_FIVE_HTR_D','H_STD_FIVE_HTR_H','H_STD_FIVE_HY','A_STD_THREE_AC','A_STD_THREE_AF','A_STD_THREE_AR','A_STD_THREE_AS','A_STD_THREE_AST','A_STD_THREE_AY','A_STD_THREE_FTAG','A_STD_THREE_FTHG','A_STD_THREE_FTR_A','A_STD_THREE_FTR_D','A_STD_THREE_FTR_H','A_STD_THREE_HC','A_STD_THREE_HF','A_STD_THREE_HR','A_STD_THREE_HS','A_STD_THREE_HST','A_STD_THREE_HTAG','A_STD_THREE_HTHG','A_STD_THREE_HTR_A','A_STD_THREE_HTR_D','A_STD_THREE_HTR_H','A_STD_THREE_HY','H_STD_THREE_AC','H_STD_THREE_AF','H_STD_THREE_AR','H_STD_THREE_AS','H_STD_THREE_AST','H_STD_THREE_AY','H_STD_THREE_FTAG','H_STD_THREE_FTHG','H_STD_THREE_FTR_A','H_STD_THREE_FTR_D','H_STD_THREE_FTR_H','H_STD_THREE_HC','H_STD_THREE_HF','H_STD_THREE_HR','H_STD_THREE_HS','H_STD_THREE_HST','H_STD_THREE_HTAG','H_STD_THREE_HTHG','H_STD_THREE_HTR_A','H_STD_THREE_HTR_D','H_STD_THREE_HTR_H','H_STD_THREE_HY','DNN_adam_all','DNN_adam_bf','DNN_sgd_all','DNN_sgd_bf','XGBoost_all','XGBoost_bf','SVM','RF_all','RF_bf','NB','MLP','LDA','KNN','ExraTree','ElasticNEt','Bagging_max_sample','Bagging_min_sample_all','Bagging_min_sample_bf','AdaBoost','GB','LR']
best_features_L2_NB = ['A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_HF', 'A_MEANS_FIVE_HS','A_MEANS_FIVE_HTR_D', 'H_MEANS_FIVE_AF', 'H_MEANS_FIVE_AY','H_MEANS_FIVE_HF', 'A_MEANS_THREE_AF', 'A_MEANS_THREE_HF','H_MEANS_THREE_AF', 'A_STD_FIVE_HF', 'H_STD_FIVE_AS','H_STD_FIVE_AST', 'H_STD_FIVE_FTHG', 'H_STD_FIVE_HTAG','H_STD_FIVE_HY', 'DNN_adam_all', 'DNN_adam_bf', 'DNN_sgd_all','DNN_sgd_bf', 'XGBoost_all', 'XGBoost_bf', 'SVM', 'RF_all','RF_bf', 'NB', 'MLP', 'LDA', 'KNN', 'ExraTree', 'ElasticNEt','Bagging_max_sample', 'Bagging_min_sample_all','Bagging_min_sample_bf', 'AdaBoost', 'GB']
best_features_L2_MLP = ['DNN_sgd_bf', 'RF_all', 'RF_bf', 'NB', 'MLP', 'LDA', 'KNN','Bagging_min_sample_bf']
best_feature_L2_ET = ['A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_HS', 'H_MEANS_FIVE_AF','H_MEANS_FIVE_HF', 'A_MEANS_THREE_AF', 'A_MEANS_THREE_HF','H_MEANS_THREE_AF', 'A_STD_FIVE_HF', 'H_STD_FIVE_AS','H_STD_FIVE_AST', 'H_STD_FIVE_HY', 'DNN_adam_all', 'DNN_adam_bf','DNN_sgd_bf', 'XGBoost_all', 'XGBoost_bf', 'SVM', 'RF_all','RF_bf', 'NB', 'MLP', 'LDA', 'KNN', 'ElasticNEt','Bagging_max_sample', 'Bagging_min_sample_bf', 'AdaBoost', 'GB']
best_features_L2_LR = ['A_MEANS_FIVE_AC','A_MEANS_FIVE_AF','A_MEANS_FIVE_AS','A_MEANS_FIVE_AST','A_MEANS_FIVE_AY','A_MEANS_FIVE_FTAG','A_MEANS_FIVE_FTR_H','A_MEANS_FIVE_HC','A_MEANS_FIVE_HF','A_MEANS_FIVE_HS','A_MEANS_FIVE_HTR_D','A_MEANS_FIVE_HTR_H','A_MEANS_FIVE_HY','H_MEANS_FIVE_AC','H_MEANS_FIVE_AF','H_MEANS_FIVE_AS','H_MEANS_FIVE_AST','H_MEANS_FIVE_AY','H_MEANS_FIVE_FTAG','H_MEANS_FIVE_FTHG','H_MEANS_FIVE_FTR_A','H_MEANS_FIVE_HC','H_MEANS_FIVE_HF','H_MEANS_FIVE_HR','H_MEANS_FIVE_HST','H_MEANS_FIVE_HTAG','H_MEANS_FIVE_HTR_H','H_MEANS_FIVE_HY','A_MEANS_THREE_AC','A_MEANS_THREE_AF','A_MEANS_THREE_AS','A_MEANS_THREE_FTHG','A_MEANS_THREE_HC','A_MEANS_THREE_HF','A_MEANS_THREE_HS','A_MEANS_THREE_HST','A_MEANS_THREE_HTR_D','A_MEANS_THREE_HY','H_MEANS_THREE_AC','H_MEANS_THREE_AF','H_MEANS_THREE_FTHG','H_MEANS_THREE_HF','H_MEANS_THREE_HS','H_MEANS_THREE_HST','H_MEANS_THREE_HY','A_STD_FIVE_AC','A_STD_FIVE_AF','A_STD_FIVE_AST','A_STD_FIVE_AY','A_STD_FIVE_FTAG','A_STD_FIVE_FTHG','A_STD_FIVE_FTR_D','A_STD_FIVE_FTR_H','A_STD_FIVE_HC','A_STD_FIVE_HF','A_STD_FIVE_HS','A_STD_FIVE_HST','A_STD_FIVE_HY','H_STD_FIVE_AC','H_STD_FIVE_AF','H_STD_FIVE_AS','H_STD_FIVE_AST','H_STD_FIVE_AY','H_STD_FIVE_FTAG','H_STD_FIVE_FTHG','H_STD_FIVE_HF','H_STD_FIVE_HS','H_STD_FIVE_HST','H_STD_FIVE_HTAG','H_STD_FIVE_HTHG','H_STD_FIVE_HY','A_STD_THREE_AF','A_STD_THREE_AS','A_STD_THREE_AST','A_STD_THREE_FTHG','A_STD_THREE_HC','A_STD_THREE_HF','A_STD_THREE_HS','A_STD_THREE_HST','H_STD_THREE_AF','H_STD_THREE_AS','H_STD_THREE_AST','H_STD_THREE_FTAG','H_STD_THREE_FTHG','H_STD_THREE_HF','H_STD_THREE_HS','H_STD_THREE_HST','H_STD_THREE_HTHG','H_STD_THREE_HY','DNN_adam_all','DNN_adam_bf','DNN_sgd_all','DNN_sgd_bf','XGBoost_all','XGBoost_bf','SVM','RF_all','RF_bf','NB','MLP','LDA','KNN','ExraTree','ElasticNEt','Bagging_max_sample','Bagging_min_sample_all','Bagging_min_sample_bf','AdaBoost','GB','LR']


# Define all different features dataset
#all_feature_L2 = ['A_MEANS_FIVE_AC','A_MEANS_FIVE_AF','A_MEANS_FIVE_AR','A_MEANS_FIVE_AS','A_MEANS_FIVE_AST','A_MEANS_FIVE_AY','A_MEANS_FIVE_FTAG','A_MEANS_FIVE_FTHG','A_MEANS_FIVE_FTR_A','A_MEANS_FIVE_FTR_D','A_MEANS_FIVE_FTR_H','A_MEANS_FIVE_HC','A_MEANS_FIVE_HF','A_MEANS_FIVE_HR','A_MEANS_FIVE_HS','A_MEANS_FIVE_HST','A_MEANS_FIVE_HTAG','A_MEANS_FIVE_HTHG','A_MEANS_FIVE_HTR_A','A_MEANS_FIVE_HTR_D','A_MEANS_FIVE_HTR_H','A_MEANS_FIVE_HY','H_MEANS_FIVE_AC','H_MEANS_FIVE_AF','H_MEANS_FIVE_AR','H_MEANS_FIVE_AS','H_MEANS_FIVE_AST','H_MEANS_FIVE_AY','H_MEANS_FIVE_FTAG','H_MEANS_FIVE_FTHG','H_MEANS_FIVE_FTR_A','H_MEANS_FIVE_FTR_D','H_MEANS_FIVE_FTR_H','H_MEANS_FIVE_HC','H_MEANS_FIVE_HF','H_MEANS_FIVE_HR','H_MEANS_FIVE_HS','H_MEANS_FIVE_HST','H_MEANS_FIVE_HTAG','H_MEANS_FIVE_HTHG','H_MEANS_FIVE_HTR_A','H_MEANS_FIVE_HTR_D','H_MEANS_FIVE_HTR_H','H_MEANS_FIVE_HY','A_MEANS_THREE_AC','A_MEANS_THREE_AF','A_MEANS_THREE_AR','A_MEANS_THREE_AS','A_MEANS_THREE_AST','A_MEANS_THREE_AY','A_MEANS_THREE_FTAG','A_MEANS_THREE_FTHG','A_MEANS_THREE_FTR_A','A_MEANS_THREE_FTR_D','A_MEANS_THREE_FTR_H','A_MEANS_THREE_HC','A_MEANS_THREE_HF','A_MEANS_THREE_HR','A_MEANS_THREE_HS','A_MEANS_THREE_HST','A_MEANS_THREE_HTAG','A_MEANS_THREE_HTHG','A_MEANS_THREE_HTR_A','A_MEANS_THREE_HTR_D','A_MEANS_THREE_HTR_H','A_MEANS_THREE_HY','H_MEANS_THREE_AC','H_MEANS_THREE_AF','H_MEANS_THREE_AR','H_MEANS_THREE_AS','H_MEANS_THREE_AST','H_MEANS_THREE_AY','H_MEANS_THREE_FTAG','H_MEANS_THREE_FTHG','H_MEANS_THREE_FTR_A','H_MEANS_THREE_FTR_D','H_MEANS_THREE_FTR_H','H_MEANS_THREE_HC','H_MEANS_THREE_HF','H_MEANS_THREE_HR','H_MEANS_THREE_HS','H_MEANS_THREE_HST','H_MEANS_THREE_HTAG','H_MEANS_THREE_HTHG','H_MEANS_THREE_HTR_A','H_MEANS_THREE_HTR_D','H_MEANS_THREE_HTR_H','H_MEANS_THREE_HY','A_STD_FIVE_AC','A_STD_FIVE_AF','A_STD_FIVE_AR','A_STD_FIVE_AS','A_STD_FIVE_AST','A_STD_FIVE_AY','A_STD_FIVE_FTAG','A_STD_FIVE_FTHG','A_STD_FIVE_FTR_A','A_STD_FIVE_FTR_D','A_STD_FIVE_FTR_H','A_STD_FIVE_HC','A_STD_FIVE_HF','A_STD_FIVE_HR','A_STD_FIVE_HS','A_STD_FIVE_HST','A_STD_FIVE_HTAG','A_STD_FIVE_HTHG','A_STD_FIVE_HTR_A','A_STD_FIVE_HTR_D','A_STD_FIVE_HTR_H','A_STD_FIVE_HY','H_STD_FIVE_AC','H_STD_FIVE_AF','H_STD_FIVE_AR','H_STD_FIVE_AS','H_STD_FIVE_AST','H_STD_FIVE_AY','H_STD_FIVE_FTAG','H_STD_FIVE_FTHG','H_STD_FIVE_FTR_A','H_STD_FIVE_FTR_D','H_STD_FIVE_FTR_H','H_STD_FIVE_HC','H_STD_FIVE_HF','H_STD_FIVE_HR','H_STD_FIVE_HS','H_STD_FIVE_HST','H_STD_FIVE_HTAG','H_STD_FIVE_HTHG','H_STD_FIVE_HTR_A','H_STD_FIVE_HTR_D','H_STD_FIVE_HTR_H','H_STD_FIVE_HY','A_STD_THREE_AC','A_STD_THREE_AF','A_STD_THREE_AR','A_STD_THREE_AS','A_STD_THREE_AST','A_STD_THREE_AY','A_STD_THREE_FTAG','A_STD_THREE_FTHG','A_STD_THREE_FTR_A','A_STD_THREE_FTR_D','A_STD_THREE_FTR_H','A_STD_THREE_HC','A_STD_THREE_HF','A_STD_THREE_HR','A_STD_THREE_HS','A_STD_THREE_HST','A_STD_THREE_HTAG','A_STD_THREE_HTHG','A_STD_THREE_HTR_A','A_STD_THREE_HTR_D','A_STD_THREE_HTR_H','A_STD_THREE_HY','H_STD_THREE_AC','H_STD_THREE_AF','H_STD_THREE_AR','H_STD_THREE_AS','H_STD_THREE_AST','H_STD_THREE_AY','H_STD_THREE_FTAG','H_STD_THREE_FTHG','H_STD_THREE_FTR_A','H_STD_THREE_FTR_D','H_STD_THREE_FTR_H','H_STD_THREE_HC','H_STD_THREE_HF','H_STD_THREE_HR','H_STD_THREE_HS','H_STD_THREE_HST','H_STD_THREE_HTAG','H_STD_THREE_HTHG','H_STD_THREE_HTR_A','H_STD_THREE_HTR_D','H_STD_THREE_HTR_H','H_STD_THREE_HY','XGBoost_all','XGBoost_bf','SVM','RF_all','RF_bf','NB','MLP','LDA','KNN','ExraTree','ElasticNEt','Bagging_max_sample','Bagging_min_sample_all','Bagging_min_sample_bf','AdaBoost','GB','LR']
#best_features_L2_NB = ['A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_HF', 'A_MEANS_FIVE_HS','A_MEANS_FIVE_HTR_D', 'H_MEANS_FIVE_AF', 'H_MEANS_FIVE_AY','H_MEANS_FIVE_HF', 'A_MEANS_THREE_AF', 'A_MEANS_THREE_HF','H_MEANS_THREE_AF', 'A_STD_FIVE_HF', 'H_STD_FIVE_AS','H_STD_FIVE_AST', 'H_STD_FIVE_FTHG', 'H_STD_FIVE_HTAG','H_STD_FIVE_HY', 'XGBoost_all', 'XGBoost_bf', 'SVM', 'RF_all','RF_bf', 'NB', 'MLP', 'LDA', 'KNN', 'ExraTree', 'ElasticNEt','Bagging_max_sample', 'Bagging_min_sample_all','Bagging_min_sample_bf', 'AdaBoost', 'GB']
#best_features_L2_MLP = ['XGBoost_all', 'RF_all', 'RF_bf', 'NB', 'MLP', 'LDA', 'KNN','Bagging_min_sample_bf']
#best_feature_L2_ET = ['A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_HS', 'H_MEANS_FIVE_AF','H_MEANS_FIVE_HF', 'A_MEANS_THREE_AF', 'A_MEANS_THREE_HF','H_MEANS_THREE_AF', 'A_STD_FIVE_HF', 'H_STD_FIVE_AS','H_STD_FIVE_AST', 'H_STD_FIVE_HY', 'XGBoost_all', 'XGBoost_bf', 'SVM', 'RF_all','RF_bf', 'NB', 'MLP', 'LDA', 'KNN', 'ElasticNEt','Bagging_max_sample', 'Bagging_min_sample_bf', 'AdaBoost', 'GB']
#best_features_L2_LR = ['A_MEANS_FIVE_AC','A_MEANS_FIVE_AF','A_MEANS_FIVE_AS','A_MEANS_FIVE_AST','A_MEANS_FIVE_AY','A_MEANS_FIVE_FTAG','A_MEANS_FIVE_FTR_H','A_MEANS_FIVE_HC','A_MEANS_FIVE_HF','A_MEANS_FIVE_HS','A_MEANS_FIVE_HTR_D','A_MEANS_FIVE_HTR_H','A_MEANS_FIVE_HY','H_MEANS_FIVE_AC','H_MEANS_FIVE_AF','H_MEANS_FIVE_AS','H_MEANS_FIVE_AST','H_MEANS_FIVE_AY','H_MEANS_FIVE_FTAG','H_MEANS_FIVE_FTHG','H_MEANS_FIVE_FTR_A','H_MEANS_FIVE_HC','H_MEANS_FIVE_HF','H_MEANS_FIVE_HR','H_MEANS_FIVE_HST','H_MEANS_FIVE_HTAG','H_MEANS_FIVE_HTR_H','H_MEANS_FIVE_HY','A_MEANS_THREE_AC','A_MEANS_THREE_AF','A_MEANS_THREE_AS','A_MEANS_THREE_FTHG','A_MEANS_THREE_HC','A_MEANS_THREE_HF','A_MEANS_THREE_HS','A_MEANS_THREE_HST','A_MEANS_THREE_HTR_D','A_MEANS_THREE_HY','H_MEANS_THREE_AC','H_MEANS_THREE_AF','H_MEANS_THREE_FTHG','H_MEANS_THREE_HF','H_MEANS_THREE_HS','H_MEANS_THREE_HST','H_MEANS_THREE_HY','A_STD_FIVE_AC','A_STD_FIVE_AF','A_STD_FIVE_AST','A_STD_FIVE_AY','A_STD_FIVE_FTAG','A_STD_FIVE_FTHG','A_STD_FIVE_FTR_D','A_STD_FIVE_FTR_H','A_STD_FIVE_HC','A_STD_FIVE_HF','A_STD_FIVE_HS','A_STD_FIVE_HST','A_STD_FIVE_HY','H_STD_FIVE_AC','H_STD_FIVE_AF','H_STD_FIVE_AS','H_STD_FIVE_AST','H_STD_FIVE_AY','H_STD_FIVE_FTAG','H_STD_FIVE_FTHG','H_STD_FIVE_HF','H_STD_FIVE_HS','H_STD_FIVE_HST','H_STD_FIVE_HTAG','H_STD_FIVE_HTHG','H_STD_FIVE_HY','A_STD_THREE_AF','A_STD_THREE_AS','A_STD_THREE_AST','A_STD_THREE_FTHG','A_STD_THREE_HC','A_STD_THREE_HF','A_STD_THREE_HS','A_STD_THREE_HST','H_STD_THREE_AF','H_STD_THREE_AS','H_STD_THREE_AST','H_STD_THREE_FTAG','H_STD_THREE_FTHG','H_STD_THREE_HF','H_STD_THREE_HS','H_STD_THREE_HST','H_STD_THREE_HTHG','H_STD_THREE_HY','XGBoost_all','XGBoost_bf','SVM','RF_all','RF_bf','NB','MLP','LDA','KNN','ExraTree','ElasticNEt','Bagging_max_sample','Bagging_min_sample_all','Bagging_min_sample_bf','AdaBoost','GB','LR']

In [9]:
# Construct layer 2
layer_2 = [
    ['NB', False, 'isotonic', GaussianNB(), best_features_L2_NB],
    ['XGBoost_all', False, 'sigmoid', XGBClassifier(learning_rate =0.01,n_estimators=572,max_depth=3,min_child_weight=5,gamma=0,subsample=0.9,colsample_bytree=0.9,objective= 'binary:logistic',nthread=4,scale_pos_weight=1,seed=15), all_feature_L2],
    ['ExtraTree', False, 'isotonic', ExtraTreesClassifier(random_state=0,n_jobs=-1,criterion='entropy',max_depth=30,max_features=None,min_samples_leaf=1,min_samples_split=16,n_estimators=350), best_feature_L2_ET],
    ['MLP', False, 'no', MLPClassifier(random_state=0,activation='identity', alpha=3, hidden_layer_sizes=(10,), max_iter=200, solver= 'adam'), best_features_L2_MLP],
    ['LR', False, 'no', LogisticRegression(random_state=0,n_jobs=-1,solver='liblinear',C=0.008,penalty='l1'), best_features_L2_LR]
]

In [10]:
# Construct layer 3
layer_3 = SGDClassifier(random_state=0,penalty='elasticnet',loss='log',alpha=0.0005,l1_ratio=0.4)

In [11]:
# Get dataset function
def get_data(season_test, training_season_nb, train_one_league_only, train_league):
    
    # DB Sqlite connection
    db = "/Users/thibaultclement/Project/ligue1-predict/src/notebook/data/db/soccer_predict.sqlite"
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    
    # Get all prematch data
    df = pd.read_sql_query("SELECT * FROM pre_matchs ORDER BY INFO_Date ASC;", conn)
    df = (df[df.columns.drop(['index'])])
    
    # Select a particular league
    if train_one_league_only:
        df = df[(df['INFO_Div'] == train_league)]
    
    # Remove all game between June (include) and October (include)
    df['INFO_Date'] = pd.to_datetime(df['INFO_Date'])
    df['INFO_Date'].dt.month
    df = df[(df['INFO_Date'].dt.month < 6) | (df['INFO_Date'].dt.month > 10)]
    
    # Keep season_test for independant testing and don't use it for training
    date_start_train_season = datetime.date(season_test-training_season_nb, 8, 1)
    date_start_test_season = datetime.date(season_test, 8, 1)
    date_end_test_season = datetime.date(season_test+1, 8, 1)
    train_df = df[(df['INFO_Date'] > date_start_train_season) & (df['INFO_Date'] < date_start_test_season)].reset_index()
    test_df = df[(df['INFO_Date'] > date_start_test_season) & (df['INFO_Date'] < date_end_test_season)].reset_index()
    
    # separate features (X) from target (y) for train and test set
    X_train_df = pd.get_dummies(train_df[raw_features])
    X_test_df = pd.get_dummies(test_df[raw_features])
    y_train_df = pd.get_dummies(train_df)[target].astype('bool_')
    y_test_df= pd.get_dummies(test_df)[target].astype('bool_')
    
    # Impute of missing values (NaN) with the mean
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
    imp = imp.fit(X_train_df)
    X_imputed = imp.transform(X_train_df)
    X_train_df = pd.DataFrame(X_imputed, columns = X_train_df.columns)
    imp = imp.fit(X_test_df) # TODO test with training fit only (inconvenient is that it would need to be save for reuse to predict)
    X_imputed = imp.transform(X_test_df)
    X_test_df = pd.DataFrame(X_imputed, columns = X_test_df.columns)
    
    return train_df, test_df, X_train_df, X_test_df, y_train_df, y_test_df

In [12]:
# Get k-fold function
def get_kf(train_df, test_df):
    ntrain = train_df.shape[0]
    ntest = test_df.shape[0]
    kf = KFold(ntrain, n_folds=NFOLDS, shuffle=True, random_state=0)
    return kf

In [13]:
# Out of Fold model training and predict test function
def get_oof(clf, x_train, x_test, y_train):
    
    # Prepare dataset
    oof_train = np.zeros((x_train.shape[0],))
    oof_test = np.zeros((x_test.shape[0],))
    oof_test_skf = np.empty((NFOLDS, x_test.shape[0]))
    
    # Loop on all fold
    for i, (train_index, test_index) in enumerate(get_kf(x_train, x_test)):
        # Split data for training and prediction
        x_tr = x_train[train_index]
        y_tr = y_train_df[train_index]
        x_te = x_train[test_index]
        # Calibrate model by fitting on K-1 fold
        clf.fit(x_tr, y_tr)
        
        # Predict the fold not use for fitting
        oof_train[test_index] = clf.predict_proba(x_te)[:,1]
        # Predict the test dataset
        oof_test_skf[i, :] = clf.predict_proba(x_test)[:,1]
    
    # Keep as test prediction the mean of all prediction done by K-fold
    oof_test[:] = oof_test_skf.mean(axis=0)
    
    # Return the prediction for train and test dataset
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [14]:
# Train model and predict test function
def train_model(preprocessing, calibration, classifier, features, X_train_model, X_test_model, y_train_model):

    # Check if we need to standardize X for this model (only for not tree based model)
    if preprocessing:
        sc_X = StandardScaler().fit(X_train_model[features])
        X_train_model = sc_X.transform(X_train_model[features])
        X_test_model = sc_X.transform(X_test_model[features])
    else:
        X_train_model = X_train_model[features].as_matrix()
        X_test_model = X_test_model[features].as_matrix()
        
    # Check if we need to recalibrate the prediction (using Plat's or Isotonic scaling)
    if calibration == 'sigmoid':
        clf = CalibratedClassifierCV(classifier, cv=4, method='sigmoid')
    elif calibration == 'isotonic':
        clf = CalibratedClassifierCV(classifier, cv=4, method='isotonic')
    elif calibration == 'no':
        clf = classifier
        
    # Obtain out-of-fold predictions for this model
    oof_train, oof_test = get_oof(clf, X_train_model, X_test_model, y_train_model)
    
    # Return the prediction for train and test dataset
    return oof_train, oof_test

In [15]:
# get prediction for a 'season' with training based on 'nb of train season' before the 'season'
def get_season_pred(league, season, nb_train_season, target, train_df, test_df, X_train_df, X_test_df, y_train_df, y_test_df):

    # Create array for saving meta features created by Layer 1 and used by Layer 2
    X_train_layer2 = np.zeros((X_train_df.shape[0], len(layer_1)))
    X_train_layer2 = pd.DataFrame(X_train_layer2, columns=[clf_name for clf_name, preprocessing, calibration, clf, features in layer_1])
    X_test_layer2 = np.zeros((X_test_df.shape[0], len(layer_1)))
    X_test_layer2 = pd.DataFrame(X_test_layer2, columns=[clf_name for clf_name, preprocessing, calibration, clf, features in layer_1])

    # Compute out-of-fold predictions for layer 1 
    for clf_name, preprocessing, calibration, classifier, features in layer_1:
        print "Layer 1 - Processing model:",clf_name
        oof_train, oof_test = train_model(preprocessing, calibration, classifier, features, X_train_df, X_test_df, y_train_df)
        X_train_layer2.loc[:, clf_name] = oof_train
        X_test_layer2.loc[:, clf_name] = oof_test
    
    # Concat meta features from layer 1 with initial features for training layer 2
    X_train_layer2_df = pd.concat([X_train_df, X_train_layer2], axis=1)
    X_test_layer2_df = pd.concat([X_test_df, X_test_layer2], axis=1)

    # Create array for saving meta features created by Layer 2 and used by Layer 3
    X_train_layer3 = np.zeros((X_train_df.shape[0], len(layer_2)))
    X_train_layer3 = pd.DataFrame(X_train_layer3, columns=[clf_name for clf_name, preprocessing, calibration, clf, features in layer_2])
    X_test_layer3 = np.zeros((X_test_df.shape[0], len(layer_2)))
    X_test_layer3 = pd.DataFrame(X_test_layer3, columns=[clf_name for clf_name, preprocessing, calibration, clf, features in layer_2])

    # Compute out-of-fold predictions for layer 2
    for clf_name, preprocessing, calibration, classifier, features in layer_2:
        print "Layer 2 - Processing model:",clf_name
        oof_train, oof_test = train_model(preprocessing, calibration, classifier, features, X_train_layer2_df, X_test_layer2_df, y_train_df)
        X_train_layer3.loc[:, clf_name] = oof_train
        X_test_layer3.loc[:, clf_name] = oof_test
        
    # Compute predictions for layer 3
    #print "Layer 3 - Processing model: ElasticNEt"
    #clf = layer_3
    #clf.fit(X_train_layer3, y_train_df)
    #train_probs = pd.Series(clf.predict_proba(X_test_layer3)[:,1])
    #test_probs = pd.Series(clf.predict_proba(X_test_layer3)[:,1])

    # Compute layer 3 by merging prediction from layer 2
    print "Layer 3 - Processing model: Weighted Sum"
    train_probs = X_train_layer3['NB']*0.3 + X_train_layer3['MLP']*0.3 + X_train_layer3['LR']*0.4
    test_probs = X_test_layer3['NB']*0.3 + X_test_layer3['MLP']*0.3 + X_test_layer3['LR']*0.4
    #train_probs = X_train_layer3['ExtraTree']*0.2 + X_train_layer3['MLP']*0.3 + X_train_layer3['LR']*0.5
    #test_probs = X_test_layer3['ExtraTree']*0.2 + X_test_layer3['MLP']*0.3 + X_test_layer3['LR']*0.5

    # Attach final prediction to informative columns of dataset to bet analysis
    result_train_info = pd.concat([train_df[column_info], train_probs.rename("PROBS"), y_train_df], axis=1)
    result_test_info = pd.concat([test_df[column_info], test_probs.rename("PROBS"), y_test_df], axis=1)
    # Attach final prediction to list of feature of dataset to full analysis
    result_train_all = pd.concat([train_df[column_info], train_probs.rename("PROBS"), y_train_df], axis=1)
    result_test_all = pd.concat([test_df[column_info], test_probs.rename("PROBS"), y_test_df], axis=1)
    # Calculate difference between my prediction and bookmaker prediction
    result_train_info['PROB_LESS_ODD'] = result_train_info['PROBS'] - result_train_info[odd_avg].apply(lambda x: 1/x)
    result_test_info['PROB_LESS_ODD'] = result_test_info['PROBS'] - result_test_info[odd_avg].apply(lambda x: 1/x)
    result_train_all['PROB_LESS_ODD'] = result_train_all['PROBS'] - result_train_all[odd_avg].apply(lambda x: 1/x)
    result_test_all['PROB_LESS_ODD'] = result_test_all['PROBS'] - result_test_all[odd_avg].apply(lambda x: 1/x)

    # Create a new columns containing the gain. If bet success it's equal to odd -1, else -1 (loose your bet)
    result_train_info['WIN_AVG'] = result_train_info[odd_avg]-1
    result_train_info['WIN_PINNACLE'] = result_train_info[odd_pinnacle]-1
    result_train_info.loc[result_train_info.INFO_FTR != bet_on, 'WIN_AVG'] = -1
    result_train_info.loc[result_train_info.INFO_FTR != bet_on, 'WIN_PINNACLE'] = -1
    result_train_all['WIN_AVG'] = result_train_info['WIN_AVG']
    result_train_all['WIN_PINNACLE'] = result_train_info['WIN_AVG']
    result_test_info['WIN_AVG'] = result_test_info[odd_avg]-1
    result_test_info['WIN_PINNACLE'] = result_test_info[odd_pinnacle]-1
    result_test_info.loc[result_test_info.INFO_FTR != bet_on, 'WIN_AVG'] = -1
    result_test_info.loc[result_test_info.INFO_FTR != bet_on, 'WIN_PINNACLE'] = -1
    result_test_all['WIN_AVG'] = result_test_info['WIN_AVG']
    result_test_all['WIN_PINNACLE'] = result_test_info['WIN_AVG']

    # Save to CSV
    #result_train_info.to_csv('./result/test/'+league+'-'+str(season)+'-'+str(nb_train_season)+'-'+target+'-train-info.csv')
    result_test_info.to_csv('./result/test/'+league+'-'+str(season)+'-'+str(nb_train_season)+'-'+target+'-test-info.csv')
    #result_train_all.to_csv('./result/test/'+league+'-'+str(season)+'-'+str(nb_train_season)+'-'+target+'-train-all.csv')
    #result_test_all.to_csv('./result/test/'+league+'-'+str(season)+'-'+str(nb_train_season)+'-'+target+'-test-all.csv')


In [16]:
# test all combination of season with number of year of training

start = datetime.datetime.now()

for league in leagues:
    for test_season in test_seasons:
        for nb_train_season in nb_train_seasons:
            print league,'test season',test_season,'training on ',nb_train_season,'historical season'
            train_df, test_df, X_train_df, X_test_df, y_train_df, y_test_df = get_data(test_season, nb_train_season, True, league)
            get_season_pred(league, test_season, nb_train_season, target, train_df, test_df, X_train_df, X_test_df, y_train_df, y_test_df)
        
print("Finish after %d seconds" % (datetime.datetime.now() - start).total_seconds())

D1 test season 2013 training on  5 historical season
Layer 1 - Processing model: DNN_adam_all
Layer 1 - Processing model: DNN_adam_bf
Layer 1 - Processing model: DNN_sgd_all
Layer 1 - Processing model: DNN_sgd_bf
Layer 1 - Processing model: XGBoost_all
Layer 1 - Processing model: XGBoost_bf
Layer 1 - Processing model: SVM
Layer 1 - Processing model: RF_all
Layer 1 - Processing model: RF_bf
Layer 1 - Processing model: NB
Layer 1 - Processing model: MLP
Layer 1 - Processing model: LDA
Layer 1 - Processing model: KNN
Layer 1 - Processing model: ExraTree
Layer 1 - Processing model: ElasticNEt
Layer 1 - Processing model: Bagging_max_sample
Layer 1 - Processing model: Bagging_min_sample_all
Layer 1 - Processing model: Bagging_min_sample_bf
Layer 1 - Processing model: AdaBoost
Layer 1 - Processing model: GB
Layer 1 - Processing model: LR
Layer 2 - Processing model: NB
Layer 2 - Processing model: XGBoost_all
Layer 2 - Processing model: ExtraTree
Layer 2 - Processing model: MLP


/anaconda/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


Layer 2 - Processing model: LR
Layer 3 - Processing model: Weighted Sum
D1 test season 2014 training on  5 historical season
Layer 1 - Processing model: DNN_adam_all
Layer 1 - Processing model: DNN_adam_bf
Layer 1 - Processing model: DNN_sgd_all
Layer 1 - Processing model: DNN_sgd_bf
Layer 1 - Processing model: XGBoost_all
Layer 1 - Processing model: XGBoost_bf
Layer 1 - Processing model: SVM
Layer 1 - Processing model: RF_all
Layer 1 - Processing model: RF_bf
Layer 1 - Processing model: NB
Layer 1 - Processing model: MLP
Layer 1 - Processing model: LDA
Layer 1 - Processing model: KNN
Layer 1 - Processing model: ExraTree
Layer 1 - Processing model: ElasticNEt
Layer 1 - Processing model: Bagging_max_sample
Layer 1 - Processing model: Bagging_min_sample_all
Layer 1 - Processing model: Bagging_min_sample_bf
Layer 1 - Processing model: AdaBoost
Layer 1 - Processing model: GB
Layer 1 - Processing model: LR
Layer 2 - Processing model: NB
Layer 2 - Processing model: XGBoost_all
Layer 2 - Proc